In [1]:
import os
import json

import pandas as pd
import numpy as np

from tqdm import tqdm
from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

In [2]:
import io
import os
import json
import distutils.dir_util
from collections import Counter

import numpy as np


def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath("./arena_data/" + parent)
    with io.open("./arena_data/" + fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)


def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)

    return json_obj


def debug_json(r):
    print(json.dumps(r, ensure_ascii=False, indent=4))


def remove_seen(seen, l):
    seen = set(seen)
    return [x for x in l if not (x in seen)]


def most_popular(playlists, col, topk_count):
    c = Counter()

    for doc in playlists:
        c.update(doc[col])

    topk = c.most_common(topk_count)
    return c, [k for k, v in topk]

In [13]:
class PlaylistEmbedding:
    def __init__(self, FILE_PATH):
        self.FILE_PATH = FILE_PATH
        self.min_count = 3
        self.size = 100
        self.window = 210
        self.sg = 5
        self.p2v_model = WordEmbeddingsKeyedVectors(self.size)
        with open(os.path.join(FILE_PATH, 'train.json'), encoding="utf-8") as f:
            self.train = json.load(f)
        with open(os.path.join(FILE_PATH, 'val.json'), encoding="utf-8") as f:
            self.val = json.load(f)
#         with open(os.path.join(FILE_PATH, 'results.json'), encoding="utf-8") as f:
#             self.most_results = json.load(f)
            
    def get_dic(self, train, val):
        song_dic = {}
        tag_dic = {}
        data = train + val
        for q in tqdm(data):
            song_dic[str(q['id'])] = q['songs']
            tag_dic[str(q['id'])] = q['tags']
        self.song_dic = song_dic
        self.tag_dic = tag_dic
        total = list(map(lambda x: list(map(str, x['songs'])) + list(x['tags']), data))
        total = [x for x in total if len(x)>1]
        self.total = total
        
    def get_w2v(self, total, min_count, size, window, sg):
        w2v_model = Word2Vec(total, min_count = min_count, size = size, window = window, sg = sg)
        self.w2v_model = w2v_model
            
    def update_p2v(self, train, val, w2v_model):
        ID = []   
        vec = []
        for q in tqdm(train + val):
            tmp_vec = 0
            if len(q['songs'])>=1:
                for song in q['songs'] + q['tags']:
                    try: 
                        tmp_vec += w2v_model.wv.get_vector(str(song))
                    except KeyError:
                        pass
            if type(tmp_vec)!=int:
                ID.append(str(q['id']))    
                vec.append(tmp_vec)
        self.p2v_model.add(ID, vec)
    
    def get_result(self, p2v_model, song_dic, tag_dic, most_results, val):
        answers = []
        for n, q in tqdm(enumerate(val), total = len(val)):
            try:
                most_id = [x[0] for x in p2v_model.most_similar(str(q['id']), topn=200)]
                get_song = []
                get_tag = []
                for ID in most_id:
                    get_song += song_dic[ID]
                    get_tag += tag_dic[ID]
                get_song = list(pd.value_counts(get_song)[:200].index)
                get_tag = list(pd.value_counts(get_tag)[:20].index)
                answers.append({
                    "id": q["id"],
                    "songs": remove_seen(q["songs"], get_song)[:100],
                    "tags": remove_seen(q["tags"], get_tag)[:10],
                })
            except:
                answers.append({
                  "id": most_results[n]["id"],
                  "songs": most_results[n]['songs'],
                  "tags": most_results[n]["tags"],
                }) 
        # check and update answer
        for n, q in enumerate(answers):
            if len(q['songs'])!=100:
                answers[n]['songs'] += remove_seen(q['songs'], self.most_results[n]['songs'])[:100-len(q['songs'])]
            if len(q['tags'])!=10:
                answers[n]['tags'] += remove_seen(q['tags'], self.most_results[n]['tags'])[:10-len(q['tags'])]  
        self.answers = answers
    
    def run(self):
        self.get_dic(self.train, self.val)
        self.get_w2v(self.total, self.min_count, self.size, self.window, self.sg)
        self.update_p2v(self.train, self.val, self.w2v_model)
        self.get_result(self.p2v_model, self.song_dic, self.tag_dic, self.most_results, self.val)
        write_json(self.answers, 'results.json')

In [14]:
FILE_PATH = './../data/'
U_space = PlaylistEmbedding(FILE_PATH)
# U_space.run()
U_space.get_dic(U_space.train, U_space.val)

100%|██████████| 138086/138086 [00:00<00:00, 471996.91it/s]


In [15]:
#U_space.get_w2v(U_space.total, U_space.min_count, U_space.size, U_space.window, U_space.sg)

# save my model
#U_space.w2v_model.save('w2v_0614.model')

In [16]:
U_space.w2v_model = Word2Vec.load('w2v_0614.model')

In [17]:
U_space.update_p2v(U_space.train, U_space.val, U_space.w2v_model)

100%|██████████| 138086/138086 [00:24<00:00, 5734.01it/s] 


In [18]:
U_space.p2v_model

In [19]:
with open(os.path.join("/Users/jonghyunlee/GitRepo/coldmelon/collaborative_filtering/", 'results.json'), encoding="utf-8") as f:
    U_space.most_results = json.load(f)

In [20]:
U_space.get_result(U_space.p2v_model, U_space.song_dic, U_space.tag_dic, U_space.most_results, U_space.val)

100%|██████████| 23015/23015 [03:17<00:00, 116.81it/s]


In [23]:
write_json(U_space.answers, 'results.json')

In [21]:
U_space.most_results[0]

{'id': 118598,
 'songs': [207912,
  623047,
  422438,
  703323,
  638488,
  322215,
  140837,
  690767,
  157900,
  39436,
  516376,
  11657,
  439161,
  78983,
  569715,
  394031,
  413459,
  213435,
  328908,
  472374,
  217622,
  262430,
  58773,
  557956,
  66475,
  207558,
  209622,
  494552,
  236393,
  607459,
  551157,
  367826,
  411438,
  292859,
  654757,
  448547,
  385871,
  387859,
  76888,
  187531,
  358186,
  326204,
  205757,
  438857,
  695032,
  542127,
  693988,
  571016,
  447762,
  140867,
  625875,
  459256,
  654428,
  620311,
  272379,
  307938,
  616144,
  32120,
  412246,
  545816,
  646988,
  572238,
  671973,
  464051,
  333595,
  570479,
  365613,
  421833,
  249378,
  55791,
  455806,
  1133,
  219415,
  284913,
  376435,
  585728,
  255937,
  112399,
  680970,
  61771,
  613740,
  400781,
  554751,
  278184,
  493762,
  224921,
  640239,
  146989,
  467269,
  5049,
  226991,
  468494,
  375776,
  424080,
  21512,
  70624,
  673026,
  679436,
  396028,
 

In [22]:
U_space.answers[0]

{'id': 118598,
 'songs': [207912,
  623047,
  703323,
  422438,
  638488,
  322215,
  157900,
  39436,
  516376,
  439161,
  690767,
  140837,
  569715,
  394031,
  413459,
  209622,
  557956,
  472374,
  411438,
  494552,
  328908,
  385871,
  78983,
  693988,
  262430,
  217622,
  607459,
  545816,
  447762,
  367826,
  448547,
  187531,
  438857,
  421833,
  382962,
  307938,
  545553,
  616144,
  66475,
  667791,
  292859,
  551157,
  620311,
  396028,
  249378,
  219415,
  226991,
  606063,
  485776,
  203153,
  1133,
  667631,
  212592,
  21512,
  58773,
  209684,
  375776,
  343420,
  412246,
  671973,
  424080,
  554509,
  169426,
  646988,
  654428,
  365613,
  619507,
  278184,
  567076,
  467269,
  182907,
  673026,
  650957,
  619259,
  259770,
  546713,
  499267,
  320642,
  530762,
  654034,
  585728,
  352807,
  680970,
  520540,
  172378,
  416417,
  693580,
  574308,
  570479,
  151704,
  407255,
  640057,
  587469,
  10690,
  314381,
  360083,
  251035,
  700090,
  34